# Notebook 01 — Data Preprocessing (PySpark / eSfinge)This notebook loads raw procurement data from the Brazilian eSfinge system, performsfeature engineering, and segments bids into four equipment-specific datasets.> **Note:** This notebook requires PySpark and the raw eSfinge data files> (HPC cluster environment). The preprocessed CSVs are already provided in `../data/`.> Run this notebook only if you need to reproduce the datasets from raw data.

In [ ]:
# Install optional dependencies (uncomment as needed for HPC environment)# !pip install unidecode# !pip install pyspark# !pip install pyarrow

In [ ]:
import osimport reimport randomimport warningsimport numpy as npimport pandas as pdimport shutilfrom unidecode import unidecodefrom scipy.stats import norm, laplace# NOTE: concatenate_files was a local helper (Utils3.py) used in the HPC environment# to merge raw split CSVs. It is not needed when using the pre-processed CSVs in ../data/# Global settingswarnings.filterwarnings('ignore')np.set_printoptions(threshold=np.inf)pd.set_option('display.max_rows', None)pd.set_option('display.max_columns', None)pd.set_option('display.max_colwidth', None)# Visualizationimport matplotlib.pyplot as pltfrom matplotlib.lines import Line2Dimport seaborn as snsfrom IPython import display%matplotlib inline# Machine learningfrom sklearn.metrics import (    accuracy_score, log_loss, mean_absolute_error, mean_squared_error, confusion_matrix,    precision_score, recall_score, roc_auc_score, brier_score_loss, f1_score,    precision_recall_fscore_support)# PySparkfrom pyspark.sql import SparkSessionfrom pyspark.sql import functions as Fimport pyspark.pandas as psfrom pyspark.sql import Windowfrom pyspark.sql.types import StringType, BooleanTypefrom pyspark.sql.functions import (    sum, mean, percentile_approx, stddev, datediff, to_timestamp, substring, lit, col, udf,    length, expr, to_date, max, min, upper, regexp_replace, isnan, when, count, dense_rank,    date_format, monotonically_increasing_id, row_number, regexp_extract, translate, log,    countDistinct, size, collect_set, concat)

In [ ]:
spark = SparkSession.builder \    .appName('SubFraudGMM') \    .config("spark.driver.memory", "1200g") \    .config("spark.executor.memory", "1000g") \    .config("spark.executor.cores", "8") \    .config("spark.driver.maxResultSize", "50g") \    .config("spark.memory.fraction", "0.8") \    .config("spark.memory.storageFraction", "0.5") \    .config("spark.sql.shuffle.partitions", "500") \    .config("spark.default.parallelism", "500") \    .config("spark.sql.files.maxPartitionBytes", "512m") \    .config("spark.sql.files.openCostInBytes", "134217728") \    .config("spark.sql.inMemoryColumnarStorage.compressed", "true") \    .config("spark.sql.inMemoryColumnarStorage.batchSize", "10000") \    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \    .config("spark.eventLog.enabled", "false") \    .config("spark.sql.debug.maxToStringFields", "100") \    .config("spark.sql.ui.retainedExecutions", "50") \    .getOrCreate()spark.sparkContext.setLogLevel("ERROR")

In [ ]:
directory_esfinge = '/mpsc/eSfinge/'diretorio_esfinge_uniGestora = '../../../../mpsc/eSfinge/Dados auxiliares/'all_files = np.array(sorted(os.listdir(directory_esfinge)))all_files_uniGestora = np.array(sorted(os.listdir(diretorio_esfinge_uniGestora)))# Identify each file group by prefixfiles_proc         = [f for f in all_files if f.startswith("ProcessoLicitatorio")]files_item         = [f for f in all_files if f.startswith("ItemLicitacao")]files_cotacao      = [f for f in all_files if f.startswith("cotacao")]files_participante = [f for f in all_files if f.startswith("ParticipanteLicitacao")]files_contrato     = [f for f in all_files if f.startswith("Contrato")]files_uniGestora   = [f for f in all_files_uniGestora if f.startswith("UnidadeGestora")]files_ente         = [f for f in all_files_uniGestora if f.startswith("Ente")]

In [ ]:
def concatenate_files_spark(spark, path, files, inferSchema=False):    files_with_path = [f"{path}/{file}" for file in files]    return spark.read.csv(files_with_path, header=True, inferSchema=inferSchema)# Load all raw data tablesdatasets = {    "proc":         (directory_esfinge, files_proc),    "cotacao":      (directory_esfinge, files_cotacao),    "participante": (directory_esfinge, files_participante),    "item":         (directory_esfinge, files_item),    "contrato":     (directory_esfinge, files_contrato),    "uniGestora":   (diretorio_esfinge_uniGestora, files_uniGestora),    "ente":         (diretorio_esfinge_uniGestora, files_ente)}df_spark = {key: concatenate_files_spark(spark, path, files) for key, (path, files) in datasets.items()}# Select relevant columns from each tabledf_spark["proc"]         = df_spark["proc"].select("ID ProcedimentoLictatorio", "numeroEdital", "Data AberturaCertame", "Data Homologacao", "ID UnidadeGestora", "Competencia")df_spark["participante"] = df_spark["participante"].select("ID ParticipanteLicitacaoCotacao", "nomeParticipante", "CPF/CNPJ Participante Cotacao")df_spark["uniGestora"]   = df_spark["uniGestora"].select("ID UnidadeGestora", "Unidade Gestora", "ID Ente")df_spark["ente"]         = df_spark["ente"].select("ID Ente", "Ente")df_spark["item"]         = df_spark["item"].select("ID ItemLicitacao", "Descrição Item Licitação", "Qtd Item Licitado", "ID ProcedimentoLictatorio")df_spark["cotacao"]      = df_spark["cotacao"].select("ID Cotacao", "Qtd Item Cotado", "Valor Total Cotado Item", "Vencedor?", "ID ParticipanteLicitacaoCotacao", "ID ItemLicitacao")df_spark["contrato"]     = df_spark["contrato"].select("ID Contrato", "DescriçãoObjetivo", "codigoCicContratado", "nomeContratado", "Data Assinatura", "Data Vencimento", "Valor Contrato", "ID ProcedimentoLictatorio")# Join all tablesdf_merged = (df_spark["proc"]    .join(df_spark["item"],         "ID ProcedimentoLictatorio", "inner")    .join(df_spark["cotacao"],      "ID ItemLicitacao",          "left")    .join(df_spark["participante"], "ID ParticipanteLicitacaoCotacao", "inner")    .join(df_spark["uniGestora"],   "ID UnidadeGestora",         "inner")    .join(df_spark["ente"],         "ID Ente",                   "inner"))df_merged = df_merged.orderBy("ID ProcedimentoLictatorio")# Type coercions and derived columnsdf_merged = (df_merged    .withColumn("Valor Total Cotado Item", regexp_replace(col("Valor Total Cotado Item"), ",", ".").cast("double"))    .withColumn("Qtd Item Licitado",       regexp_replace(col("Qtd Item Licitado"),       ",", ".").cast("double"))    .withColumn("Qtd Item Cotado",         regexp_replace(col("Qtd Item Cotado"),         ",", ".").cast("double"))    .withColumn("unit_price", (col("Valor Total Cotado Item") / col("Qtd Item Licitado"))))# Remove accents and convert item descriptions to upper caseudf_unidecode = udf(lambda x: unidecode(x) if x else None, StringType())df_merged = df_merged.withColumn("Descrição Item Licitação", upper(udf_unidecode(col("Descrição Item Licitação"))))# Cast column typesdf_merged = (df_merged    .withColumn("ID ItemLicitacao",                col("ID ItemLicitacao").cast("int"))    .withColumn("ID UnidadeGestora",               col("ID UnidadeGestora").cast("int"))    .withColumn("ID ProcedimentoLictatorio",       col("ID ProcedimentoLictatorio").cast("int"))    .withColumn("ID ParticipanteLicitacaoCotacao", col("ID ParticipanteLicitacaoCotacao").cast("int"))    .withColumn("Vencedor?",                       col("Vencedor?").cast("float"))    .withColumn("CPF/CNPJ Participante Cotacao",   substring(col("CPF/CNPJ Participante Cotacao"), 1, 8).cast("int"))    .withColumn("Data Homologacao",    to_timestamp("Data Homologacao",    "dd/MM/yyyy HH:mm:ss"))    .withColumn("Data AberturaCertame", to_timestamp("Data AberturaCertame", "dd/MM/yyyy HH:mm:ss")))# Filter invalid CNPJsdf_merged = df_merged.filter(col("CPF/CNPJ Participante Cotacao") > 1)# Year columndf_merged = df_merged.withColumn("Ano", substring(col("Competencia"), 1, 4).cast("int"))# Remove bids where homologation precedes opening (data inconsistency)df_merged = df_merged.filter(col("Data Homologacao") >= col("Data AberturaCertame"))# Fraud item IDs (from Operação Patrola investigation)item_fraud_id = [11584542, 12979877, 16178479, 13689385, 11744285, 10027272, 11346035, 8936878,                 10761307, 11977346, 14203304, 11361724, 14348037, 11397364, 10076961, 12298749,                 10257914, 10962118, 14414964, 9554871, 11263631, 12290513, 12290514, 11073284,                 10582146, 12024294, 14247372, 14247371, 12128720, 12128719, 14254560, 12092337,                 10230106, 12106697, 12522349, 13647673, 10525709, 11608487, 12776209, 9855100,                 14337365, 14433870, 13886171, 13565028, 12193860, 12227028, 11240922, 13753389,                 13753390, 14666244, 14666245, 13559660, 9867739, 8745233, 10050334, 14840641,                 11590181, 9696347, 14488880, 13713177, 15663110, 11458763, 15667962, 12329952,                 14074796, 14074795, 10474472, 14374814, 14159821, 8915092, 14365204, 16346733,                 10109167, 14706469, 14297125, 13115054, 12306656, 16578215, 9937877, 12144806,                 12144807, 14403083, 12170281, 9983089, 10060783, 9430160, 11576784, 12073653,                 14666808, 12371307, 13592022, 13547537, 12237131, 10235662, 12479764, 10259630,                 14225203, 14203437, 13962911]# Label fraud: winning bid on a fraud item = frauddf_merged = df_merged.withColumn("fraude", (    (col("Vencedor?") == -1.0) & col("ID ItemLicitacao").isin(item_fraud_id)).cast("int"))df_merged = df_merged.dropna()df_merged = df_merged.select(    "ID ProcedimentoLictatorio", "ID ItemLicitacao", "ID ParticipanteLicitacaoCotacao",    "ID UnidadeGestora", "numeroEdital", "Descrição Item Licitação", "Ente",    "Qtd Item Licitado", "Valor Total Cotado Item", "Data Homologacao", "Data AberturaCertame",    "CPF/CNPJ Participante Cotacao", "nomeParticipante", "Vencedor?", "fraude", "Ano", "unit_price")

## Feature Engineering

In [ ]:
# Number of bidding participants per procurement processdf_participantes = df_merged.groupBy("ID ProcedimentoLictatorio").agg(    countDistinct("CPF/CNPJ Participante Cotacao").alias("num_partic"))df_merged = df_merged.join(df_participantes, on="ID ProcedimentoLictatorio", how="left")# win — Proportion of winning proposals for a supplier within a managing unitwindowSpec_numerador   = Window.partitionBy("CPF/CNPJ Participante Cotacao", "ID UnidadeGestora").orderBy("Data Homologacao").rowsBetween(Window.unboundedPreceding, Window.currentRow)windowSpec_denominador = Window.partitionBy("CPF/CNPJ Participante Cotacao").orderBy("Data Homologacao").rowsBetween(Window.unboundedPreceding, Window.currentRow)df_vencedores = df_merged.withColumn("auction_winner_flag", when(col("Vencedor?") == -1, lit(1)).otherwise(lit(0)))df_vencedores = df_vencedores.withColumn("win", (    sum("auction_winner_flag").over(windowSpec_numerador) /    sum("auction_winner_flag").over(windowSpec_denominador)))df_merged = df_vencedores.na.fill({'win': 0})# met — Whether the supplier has previously contracted with the managing unitwindowSpec = Window.partitionBy("ID UnidadeGestora", "CPF/CNPJ Participante Cotacao").orderBy("Data AberturaCertame")df_merged = df_merged.withColumn("met", when(dense_rank().over(windowSpec) > 1, 1).otherwise(0))# num — Number of procurement processes the supplier has participated inwindowSpecCount = Window.partitionBy('CPF/CNPJ Participante Cotacao').orderBy('Data AberturaCertame').rowsBetween(Window.unboundedPreceding, Window.currentRow)df_merged = df_merged.withColumn('num', F.approx_count_distinct('ID ProcedimentoLictatorio').over(windowSpecCount))# period — Active lifespan of the supplier in the dataset (days)windowSpec = Window.partitionBy('CPF/CNPJ Participante Cotacao').orderBy('Data AberturaCertame').rowsBetween(Window.unboundedPreceding, Window.currentRow)df_merged = df_merged.withColumn('period', (    max("Data AberturaCertame").over(windowSpec).cast('long') -    min("Data AberturaCertame").over(windowSpec).cast('long')) / (3600 * 24))# duration — Duration of each procurement process (days)df_merged = df_merged.withColumn("duration", datediff(col("Data Homologacao"), col("Data AberturaCertame")))# unique — Proportion of unique winners among managing unit procurementswindowSpec = Window.partitionBy("ID UnidadeGestora").orderBy("Data Homologacao").rowsBetween(Window.unboundedPreceding, Window.currentRow)df_merged = df_merged.withColumn("unique",    size(collect_set(when(col("Vencedor?") == -1, col("CPF/CNPJ Participante Cotacao"))).over(windowSpec)) /    sum(when(col("Vencedor?") == -1, 1)).over(windowSpec))df_merged = df_merged.dropna()

## Dataset Segmentation by Equipment Type

In [ ]:
def move_and_rename_file(source_dir, target_dir, new_filename):    """Move the first part-* CSV file from source_dir to target_dir/new_filename."""    csv_files = [f for f in os.listdir(source_dir) if f.startswith("part-")]    if csv_files:        old_file = os.path.join(source_dir, csv_files[0])        new_file = os.path.join(target_dir, new_filename)        shutil.move(old_file, new_file)        print(f"File renamed to {new_file}")

### Track-type tractor (trator de esteira)

In [ ]:
# Known fraud records for track-type tractorsfraude_trator = df_merged.filter(col("fraude") == 1)itens = ["TRATOR", "ESTEIRA"]fraude_trator = fraude_trator.filter(    (col("Descrição Item Licitação").rlike(r"\b" + itens[0] + r"\b")) &    (col("Descrição Item Licitação").rlike(itens[1])))# All bids matching the item descriptiondf_filtered = df_merged.filter(    (col("Descrição Item Licitação").rlike(r"\b" + itens[0] + r"\b")) &    (col("Descrição Item Licitação").rlike(itens[1])))# Filter by minimum value and exclude spare-parts / service keywordsdf_filtered_teste = df_filtered.filter(col("Valor Total Cotado Item") > 5000)palavras_exclusao = ["CARREGADEIRA", "AGRICOLA", "ESCAVACAO", "ESCARIFICADOR", "COTA", "PISTAO",    "COLMEIA", "BRITADO", "CARGA", "BANCO", "PERFURADOR", "PESADOS", "VEICULO", "ALTERNADOR",    "ARRANQUE", "CARCACA", "EXTENSAO", "ESPALHAMENTO", "ESTRADA", "PONTA", "CENTRAL", "REVISAO",    "REMOCAO", "GAXETA", "RECAUCHUTAGEM", "DESPESA", "SENSOR", "REGULARIZACAO", "TRAVESSA",    "MATERIAL", "LIMP", "PROTETOR", "PESADAS", "LAMPADA", "BATERIA", "HELICE", "PATROLA",    "LAVACAO", "CILINDRO", "CAMARA", "MECANICA", "ELETRICA", "HORA", "RODA", "KIT", "CABECOTE",    "SAPO", "RIPPER", "MESTRE", "LANCA", "VIDRO", "FROTA", "CODIGO", "LEANTE", "ANEIS",    "DENTADA", "CONTROLLER", "AGUA", "TORQUE", "CONJUNTO", "BREQUIM", "CAMISA", "REFERENTE",    "BRITADOR", "BRONZINA", "MANGOTE", "FREIO", "REGULADOR", "ALIEN", "REVISAR", "ROMPEDOR",    "RODOVIARIO", "FORNECIMENTO", "TRASEIRA", "DIANTEIRA", "OLHAL", "CHAPA", "LAVAGEM",    "CREMALHEIRA", "PARABRISA", "ROSCA", "MANOMETRO", "TERMOMETRO", "TERRAPLANAGEM",    "EMBREAGEM", "MATERIAIS", "SECRETARIA", "AGRICULTURA", "RECUP", "MENSAL", "ESCAVACOES",    "TERRAPLENAGEM", "OFICINA", "LEILAO", "CABINE", "MANUTENCAO", "SERV", "PECA", "REFORMA",    "LAMINA", "MAO", "LIXO", "PNEU", "FACA", "RECUPERACAO", "FILTRO", "DENTE", "PARAFUSO",    "UNHA", "JOGO", "PORCA", "CORRENTE", "HORIMETRO", "TAMPA", "MANCAL", "RETENTOR", "SAPATA",    "PORTA", "LUVA", "BUCHA", "BOMBA", "ANEL", "ROLAMENTO", "ARRUELA", "COLAR", "EIXO",    "JUNTA", "DISCO", "CONSERTO", "PLACA", "TAMBOR", "FIXADOR", "VEDADOR", "GUARNICAO",    "LONA", "SEGMENTO", "GARFO", "ENGRENAGEM", "PINO", "CUBO", "LANTERNA", "ESPACADOR",    "CALCO", "VARETA", "MOLA", "ELETRODO", "ESFERA", "ARVORE", "BRACO", "BRACADEIRA",    "BARRA", "TURBINA", "ROLETE", "PINHAO", "CARDAO", "FLANGE", "LOCACAO", "VALVULA",    "MINI", "PERCUSSAO", "INJETORA", "CONCHA", "COLHEDORA", "ADUBO", "CARRETA", "ROCADEIRA",    "PLAINA", "ENXADA", "BICO", "RADIADOR", "TRAVA", "ALIENACAO", "ANCINHO", "JOYSTICK",    "CONTROLE", "CACAMBA", "PLANTADEIRA", "BATEDEIRA", "PULVERIZADOR", "PAINEL", "GRAMA",    "ENGATE", "EMPILHADEIRA", "COLETOR", "OLEO", "COMPRESSOR", "TIRA", "DIAGNOSTICO",    "PRENSA", "AMORTECEDOR", "DISTRIBUIDOR", "COMANDO", "REDUTOR", "RETIFICA", "CONTRATACAO",    "TRANSMISSAO", "PASTILHA", "COXIM", "PATIM", "POLIA", "PINTURA", "CAIXA", "SEGURO",    "REVISOES", "SEGADEIRA", "EXTRATOR", "CORREIA"]palavras_exclusao_upper = [p.upper() for p in palavras_exclusao]df_filtered_teste = df_filtered_teste.filter(~col("Descrição Item Licitação").rlike("|".join(palavras_exclusao_upper)))df_filtered_teste = df_filtered_teste.filter((col("Ano") >= 2009) & (col("Ano") <= 2024))trator = df_filtered_teste.union(fraude_trator)df_filtered_teste = trator.dropDuplicates()df_filtered_teste = df_filtered_teste.filter(col("Vencedor?") == -1)window_spec = Window.orderBy(col("Vencedor?"))df_filtered_teste = df_filtered_teste.withColumn("ID", row_number().over(window_spec))df_filtered_teste = df_filtered_teste.withColumn("ID", concat(lit("TRA"), col("ID").cast("string")))colunas = ["ID"] + [c for c in df_filtered_teste.columns if c != "ID"]df_filtered_teste = df_filtered_teste.select(colunas)df_filtered_teste.repartition(1).write.csv('../data/trator_esteira.csv', header=True, mode='overwrite')move_and_rename_file("../data/trator_esteira.csv", "../data", "trator_esteira_final.csv")

### Compaction roller (rolo compactador)

In [ ]:
# Known fraud records for compaction rollersfraude_compactador = df_merged.filter(col("fraude") == 1)itens = ["ROLO", "CTADOR"]fraude_compactador = fraude_compactador.filter(col("Descrição Item Licitação").rlike(itens[1]))df_filtered = df_merged.filter(    (col("Descrição Item Licitação").rlike(r"\b" + itens[0] + r"\b")) &    (col("Descrição Item Licitação").rlike(itens[1])))df_filtered_teste = df_filtered.filter(col("Valor Total Cotado Item") > 5000)palavras_exclusao_rolo = ["HORA", "CILINDRO", "CAMINHAO", "RECOMPOSICAO", "SUCATA", "BOBINA",    "MOTONIVELADORA", "TRANSPORTE", "PATRIMO", "PLASTICA", "PARTIDA", "TRATOR", "BALNEARIO",    "DESTORROADOR", "PEDRAS", "TERCEIRIZACAO", "MARTELO", "REATERRO", "ELO", "BIELA", "ATERRO",    "VIBROCOMPACTADOR", "COMPACTACAO", "PECA", "PAVIMENTO", "CONCRETO", "ALUGUEL", "IMPLEMENTO",    "CABECA", "MOTOR", "RECAPE", "AGRICOLA", "ESCAVACAO", "ESCARIFICADOR", "COTA", "PISTAO",    "COLMEIA", "BRITADO", "CARGA", "BANCO", "PERFURADOR", "PESADOS", "VEICULO", "ALTERNADOR",    "ARRANQUE", "CARCACA", "EXTENSAO", "ESPALHAMENTO", "ESTRADA", "PONTA", "CENTRAL", "REVISAO",    "REMOCAO", "GAXETA", "RECAUCHUTAGEM", "DESPESA", "SENSOR", "REGULARIZACAO", "TRAVESSA",    "MATERIAL", "LIMP", "PROTETOR", "PESADAS", "LAMPADA", "BATERIA", "HELICE", "PATROLA",    "LAVACAO", "CAMARA", "MECANICA", "ELETRICA", "KIT", "CABECOTE", "SAPO", "RIPPER",    "MESTRE", "LANCA", "VIDRO", "FROTA", "CODIGO", "LEANTE", "ANEIS", "DENTADA",    "CONTROLLER", "AGUA", "TORQUE", "CONJUNTO", "BREQUIM", "CAMISA", "REFERENTE", "BRITADOR",    "BRONZINA", "MANGOTE", "FREIO", "REGULADOR", "ALIEN", "REVISAR", "ROMPEDOR", "RODOVIARIO",    "FORNECIMENTO", "TRASEIRA", "DIANTEIRA", "OLHAL", "CHAPA", "LAVAGEM", "CREMALHEIRA",    "PARABRISA", "ROSCA", "MANOMETRO", "TERMOMETRO", "TERRAPLANAGEM", "EMBREAGEM", "MATERIAIS",    "SECRETARIA", "AGRICULTURA", "RECUP", "MENSAL", "ESCAVACOES", "TERRAPLENAGEM", "OFICINA",    "LEILAO", "CABINE", "MANUTENCAO", "SERV", "REFORMA", "LAMINA", "MAO", "LIXO", "PNEU",    "FACA", "RECUPERACAO", "FILTRO", "DENTE", "PARAFUSO", "UNHA", "JOGO", "PORCA", "CORRENTE",    "HORIMETRO", "TAMPA", "MANCAL", "RETENTOR", "SAPATA", "PORTA", "LUVA", "BUCHA", "BOMBA",    "ANEL", "ROLAMENTO", "ARRUELA", "COLAR", "EIXO", "JUNTA", "DISCO", "CONSERTO", "PLACA",    "TAMBOR", "FIXADOR", "VEDADOR", "GUARNICAO", "LONA", "SEGMENTO", "GARFO", "ENGRENAGEM",    "PINO", "CUBO", "LANTERNA", "ESPACADOR", "CALCO", "VARETA", "MOLA", "ELETRODO", "ESFERA",    "ARVORE", "BRACO", "BRACADEIRA", "BARRA", "TURBINA", "ROLETE", "PINHAO", "CARDAO",    "FLANGE", "LOCACAO", "VALVULA", "MINI", "PERCUSSAO", "INJETORA", "CONCHA", "COLHEDORA",    "ADUBO", "CARRETA", "ROCADEIRA", "PLAINA", "ENXADA", "BICO", "RADIADOR", "MAO", "TRAVA",    "ALIENACAO", "ANCINHO", "JOYSTICK", "CONTROLE", "CACAMBA", "PLANTADEIRA", "BATEDEIRA",    "PULVERIZADOR", "PAINEL", "GRAMA", "ENGATE", "EMPILHADEIRA", "COLETOR", "OLEO",    "COMPRESSOR", "TIRA", "DIAGNOSTICO", "PRENSA", "AMORTECEDOR", "DISTRIBUIDOR", "COMANDO",    "REDUTOR", "RETIFICA", "CONTRATACAO", "TRANSMISSAO", "PASTILHA", "COXIM", "PATIM",    "POLIA", "PINTURA", "CAIXA", "SEGURO", "REVISOES", "SEGADEIRA", "EXTRATOR", "CORREIA"]palavras_exclusao_upper = [p.upper() for p in palavras_exclusao_rolo]df_filtered_teste = df_filtered_teste.filter(~col("Descrição Item Licitação").rlike("|".join(palavras_exclusao_upper)))df_filtered_teste = df_filtered_teste.filter((col("Ano") >= 2009) & (col("Ano") <= 2024))compactador = df_filtered_teste.union(fraude_compactador)df_filtered_teste = compactador.dropDuplicates()df_filtered_teste = df_filtered_teste.filter(col("Vencedor?") == -1)window_spec = Window.orderBy(col("Vencedor?"))df_filtered_teste = df_filtered_teste.withColumn("ID", row_number().over(window_spec))df_filtered_teste = df_filtered_teste.withColumn("ID", concat(lit("ROL"), col("ID").cast("string")))colunas = ["ID"] + [c for c in df_filtered_teste.columns if c != "ID"]df_filtered_teste = df_filtered_teste.select(colunas)df_filtered_teste.repartition(1).write.csv('../data/rolo_compactador.csv', header=True, mode='overwrite')move_and_rename_file("../data/rolo_compactador.csv", "../data", "rolo_compactador_final.csv")

### Hydraulic excavator (escavadeira hidráulica)

In [ ]:
# Known fraud records for hydraulic excavatorsfraude_escavadeira = df_merged.filter(col("fraude") == 1)itens = ["ESCAVADEIRA", "HIDRAULICA"]fraude_escavadeira = fraude_escavadeira.filter(    col("Descrição Item Licitação").rlike(r"\b" + itens[0] + r"\b"))df_filtered = df_merged.filter(    col("Descrição Item Licitação").rlike(r"\b" + itens[0] + r"\b"))df_filtered_teste = df_filtered.filter(col("Valor Total Cotado Item") > 5000)palavras_exclusao_esc = ["TRANSPORTE", "PATRIMO", "PLASTICA", "PARTIDA", "TRATOR", "BALNEARIO",    "DESTORROADOR", "PEDRAS", "TERCEIRIZACAO", "MARTELO", "CILINDRO", "REATERRO", "ELO",    "RODA", "BIELA", "ATERRO", "VIBROCOMPACTADOR", "COMPACTACAO", "HORA", "PECA", "PAVIMENTO",    "CONCRETO", "ALUGUEL", "IMPLEMENTO", "CABECA", "MOTOR", "RECAPE", "CARREGADEIRA",    "AGRICOLA", "ESCARIFICADOR", "COTA", "PISTAO", "COLMEIA", "BRITADO", "CARGA", "BANCO",    "PERFURADOR", "PESADOS", "VEICULO", "ALTERNADOR", "ARRANQUE", "CARCACA", "EXTENSAO",    "ESPALHAMENTO", "ESTRADA", "PONTA", "CENTRAL", "REVISAO", "REMOCAO", "GAXETA",    "RECAUCHUTAGEM", "DESPESA", "SENSOR", "REGULARIZACAO", "TRAVESSA", "MATERIAL", "LIMP",    "PROTETOR", "PESADAS", "LAMPADA", "BATERIA", "HELICE", "PATROLA", "LAVACAO", "CAMARA",    "MECANICA", "ELETRICA", "KIT", "CABECOTE", "SAPO", "RIPPER", "MESTRE", "LANCA",    "VIDRO", "FROTA", "CODIGO", "LEANTE", "ANEIS", "DENTADA", "CONTROLLER", "AGUA",    "TORQUE", "CONJUNTO", "BREQUIM", "CAMISA", "REFERENTE", "BRITADOR", "BRONZINA",    "MANGOTE", "FREIO", "REGULADOR", "ALIEN", "REVISAR", "ROMPEDOR", "RODOVIARIO",    "FORNECIMENTO", "TRASEIRA", "DIANTEIRA", "OLHAL", "CHAPA", "LAVAGEM", "CREMALHEIRA",    "PARABRISA", "ROSCA", "MANOMETRO", "TERMOMETRO", "TERRAPLANAGEM", "EMBREAGEM",    "MATERIAIS", "SECRETARIA", "AGRICULTURA", "RECUP", "MENSAL", "TERRAPLENAGEM",    "OFICINA", "LEILAO", "CABINE", "MANUTENCAO", "SERV", "REFORMA", "LAMINA", "MAO",    "LIXO", "PNEU", "FACA", "RECUPERACAO", "FILTRO", "DENTE", "PARAFUSO", "UNHA", "JOGO",    "PORCA", "CORRENTE", "HORIMETRO", "TAMPA", "MANCAL", "RETENTOR", "SAPATA", "PORTA",    "LUVA", "BUCHA", "BOMBA", "ANEL", "ROLAMENTO", "ARRUELA", "COLAR", "EIXO", "JUNTA",    "DISCO", "CONSERTO", "PLACA", "TAMBOR", "FIXADOR", "VEDADOR", "GUARNICAO", "LONA",    "SEGMENTO", "GARFO", "ENGRENAGEM", "PINO", "CUBO", "LANTERNA", "ESPACADOR", "CALCO",    "VARETA", "MOLA", "ELETRODO", "ESFERA", "ARVORE", "BRACO", "BRACADEIRA", "BARRA",    "TURBINA", "ROLETE", "PINHAO", "CARDAO", "FLANGE", "LOCACAO", "VALVULA", "MINI",    "PERCUSSAO", "INJETORA", "CONCHA", "COLHEDORA", "ADUBO", "CARRETA", "ROCADEIRA",    "PLAINA", "ENXADA", "BICO", "RADIADOR", "TRAVA", "ALIENACAO", "ANCINHO", "JOYSTICK",    "CONTROLE", "CACAMBA", "PLANTADEIRA", "BATEDEIRA", "PULVERIZADOR", "PAINEL", "GRAMA",    "ENGATE", "EMPILHADEIRA", "COLETOR", "OLEO", "COMPRESSOR", "TIRA", "DIAGNOSTICO",    "PRENSA", "AMORTECEDOR", "DISTRIBUIDOR", "COMANDO", "REDUTOR", "RETIFICA",    "CONTRATACAO", "TRANSMISSAO", "PASTILHA", "COXIM", "PATIM", "POLIA", "PINTURA",    "CAIXA", "SEGURO", "REVISOES", "SEGADEIRA", "EXTRATOR", "CORREIA"]palavras_exclusao_upper = [p.upper() for p in palavras_exclusao_esc]df_filtered_teste = df_filtered_teste.filter(~col("Descrição Item Licitação").rlike("|".join(palavras_exclusao_upper)))df_filtered_teste = df_filtered_teste.filter((col("Ano") >= 2009) & (col("Ano") <= 2024))escavadeira = df_filtered_teste.union(fraude_escavadeira)df_filtered_teste = escavadeira.dropDuplicates()df_filtered_teste = df_filtered_teste.filter(col("Vencedor?") == -1)window_spec = Window.orderBy(col("Vencedor?"))df_filtered_teste = df_filtered_teste.withColumn("ID", row_number().over(window_spec))df_filtered_teste = df_filtered_teste.withColumn("ID", concat(lit("ESC"), col("ID").cast("string")))colunas = ["ID"] + [c for c in df_filtered_teste.columns if c != "ID"]df_filtered_teste = df_filtered_teste.select(colunas)df_filtered_teste.repartition(1).write.csv('../data/escavadeira.csv', header=True, mode='overwrite')move_and_rename_file("../data/escavadeira.csv", "../data", "escavadeira_final.csv")

### Motor grader (motoniveladora)

In [ ]:
# Known fraud records for motor gradersfraude_motoniveladora = df_merged.filter(col("fraude") == 1)itens = ["MOTONIVELADORA"]fraude_motoniveladora = fraude_motoniveladora.filter(    col("Descrição Item Licitação").rlike(r"\b" + itens[0] + r"\b"))df_filtered = df_merged.filter(    col("Descrição Item Licitação").rlike(r"\b" + itens[0] + r"\b"))df_filtered_teste = df_filtered.filter(col("Valor Total Cotado Item") > 5000)palavras_exclusao_moto = ["REVISOES", "ENGRENAGE", "RODA", "SUCATA", "CILINDRO", "BOBINA",    "TRANSPORTE", "PATRIMO", "PLASTICA", "PARTIDA", "TRATOR", "BALNEARIO", "DESTORROADOR",    "PEDRAS", "TERCEIRIZACAO", "MARTELO", "REATERRO", "ELO", "BIELA", "ATERRO",    "VIBROCOMPACTADOR", "COMPACTACAO", "HORA", "PECA", "PAVIMENTO", "CONCRETO", "ALUGUEL",    "IMPLEMENTO", "CABECA", "MOTOR", "RECAPE", "AGRICOLA", "ESCARIFICADOR", "COTA",    "PISTAO", "COLMEIA", "BRITADO", "CARGA", "BANCO", "PERFURADOR", "PESADOS", "VEICULO",    "ALTERNADOR", "ARRANQUE", "CARCACA", "EXTENSAO", "ESPALHAMENTO", "ESTRADA", "PONTA",    "CENTRAL", "REVISAO", "REMOCAO", "GAXETA", "RECAUCHUTAGEM", "DESPESA", "SENSOR",    "REGULARIZACAO", "TRAVESSA", "MATERIAL", "LIMP", "PROTETOR", "PESADAS", "LAMPADA",    "BATERIA", "HELICE", "PATROLA", "LAVACAO", "CAMARA", "MECANICA", "ELETRICA", "KIT",    "CABECOTE", "SAPO", "RIPPER", "MESTRE", "LANCA", "VIDRO", "FROTA", "CODIGO", "LEANTE",    "ANEIS", "DENTADA", "CONTROLLER", "AGUA", "TORQUE", "CONJUNTO", "BREQUIM", "CAMISA",    "REFERENTE", "BRITADOR", "BRONZINA", "MANGOTE", "FREIO", "REGULADOR", "ALIEN",    "REVISAR", "ROMPEDOR", "RODOVIARIO", "FORNECIMENTO", "TRASEIRA", "DIANTEIRA", "OLHAL",    "CHAPA", "LAVAGEM", "CREMALHEIRA", "PARABRISA", "ROSCA", "MANOMETRO", "TERMOMETRO",    "TERRAPLANAGEM", "EMBREAGEM", "MATERIAIS", "SECRETARIA", "AGRICULTURA", "RECUP",    "MENSAL", "TERRAPLENAGEM", "OFICINA", "LEILAO", "CABINE", "MANUTENCAO", "SERV",    "REFORMA", "LAMINA", "MAO", "LIXO", "PNEU", "FACA", "RECUPERACAO", "FILTRO", "DENTE",    "PARAFUSO", "UNHA", "JOGO", "PORCA", "CORRENTE", "HORIMETRO", "TAMPA", "MANCAL",    "RETENTOR", "SAPATA", "PORTA", "LUVA", "BUCHA", "BOMBA", "ANEL", "ROLAMENTO",    "ARRUELA", "COLAR", "EIXO", "JUNTA", "DISCO", "CONSERTO", "PLACA", "TAMBOR",    "FIXADOR", "VEDADOR", "GUARNICAO", "LONA", "SEGMENTO", "GARFO", "ENGRENAGEM",    "PINO", "CUBO", "LANTERNA", "ESPACADOR", "CALCO", "VARETA", "MOLA", "ELETRODO",    "ESFERA", "ARVORE", "BRACO", "BRACADEIRA", "BARRA", "TURBINA", "ROLETE", "PINHAO",    "CARDAO", "FLANGE", "LOCACAO", "VALVULA", "MINI", "PERCUSSAO", "INJETORA", "CONCHA",    "COLHEDORA", "ADUBO", "CARRETA", "ROCADEIRA", "PLAINA", "ENXADA", "BICO", "RADIADOR",    "TRAVA", "ALIENACAO", "ANCINHO", "JOYSTICK", "CONTROLE", "CACAMBA", "PLANTADEIRA",    "BATEDEIRA", "PULVERIZADOR", "PAINEL", "GRAMA", "ENGATE", "EMPILHADEIRA", "COLETOR",    "OLEO", "COMPRESSOR", "TIRA", "DIAGNOSTICO", "PRENSA", "AMORTECEDOR", "DISTRIBUIDOR",    "COMANDO", "REDUTOR", "RETIFICA", "CONTRATACAO", "TRANSMISSAO", "PASTILHA", "COXIM",    "PATIM", "POLIA", "PINTURA", "CAIXA", "SEGURO", "REVISOES", "SEGADEIRA", "EXTRATOR",    "CORREIA"]palavras_exclusao_upper = [p.upper() for p in palavras_exclusao_moto]df_filtered_teste = df_filtered_teste.filter(~col("Descrição Item Licitação").rlike("|".join(palavras_exclusao_upper)))df_filtered_teste = df_filtered_teste.filter((col("Ano") >= 2009) & (col("Ano") <= 2024))motoniveladora = df_filtered_teste.union(fraude_motoniveladora)df_filtered_teste = motoniveladora.dropDuplicates()df_filtered_teste = df_filtered_teste.filter(col("Vencedor?") == -1)window_spec = Window.orderBy(col("Vencedor?"))df_filtered_teste = df_filtered_teste.withColumn("ID", row_number().over(window_spec))df_filtered_teste = df_filtered_teste.withColumn("ID", concat(lit("MOT"), col("ID").cast("string")))colunas = ["ID"] + [c for c in df_filtered_teste.columns if c != "ID"]df_filtered_teste = df_filtered_teste.select(colunas)df_filtered_teste.repartition(1).write.csv('../data/motoniveladora.csv', header=True, mode='overwrite')move_and_rename_file("../data/motoniveladora.csv", "../data", "motoniveladora_final.csv")